In [1]:
mes = "04"

In [2]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode

#tempo
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
browser = webdriver.Firefox()

In [4]:
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [5]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [6]:
#click setor/profissional
browser.find_element("xpath", '//*[@id="s2id_autogen1"]/a/span[1]').click()
#click [profissional]
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()

In [41]:
df_backup = pd.read_excel('df_gabs2.xlsx')

df_backup = df_backup.sort_values(['nome', 'data'])
df_backup.reset_index(drop=True, inplace=True)

df = df_backup.copy()

df.head()

,chat,dia,nome,mensagem,controle,data,unidade,extra,ENTRADA,SAIDA,DURAÇÃO,PERIODO,setor,sem_ou_fds
0,[27/04/2023 23:05:15] Alexandre Bernardino: 27...,27/04/2023,ALEXANDRE BERNARDINO,27/04 - Aviccena - Alexandre - entrada 05:30,PROBS,27/04/2023,AVICCENA,NaN,05:30,07:00,01:30,6h,SEMANA - 6HS,semana
1,[27/04/2023 16:26:44] Cláudio Mueller Kakuda: ...,27/04/2023,CLAUDIO MUELLER KAKUDA,27/04 Maternidade Kakuda saida 16:15,PROBS,27/04/2023,MATERNIDADE,NaN,13:00,16:30,03:30,tarde,SEMANA - TARDE - C.CIRURGICO,semana
2,[28/04/2023 06:53:12] Emerson Domingos da Cost...,28/04/2023,EMERSON DOMINGOS DA COSTA,28/04 Osasco Emerson saída 20:20,PROBS,28/04/2023,OSASCO,NaN,19:00,20:30,01:30,noturno saida,SEMANA - NOTURNO - SAIDA,semana
3,[27/04/2023 16:44:16] Estanrley Barcelos Pinto...,27/04/2023,ESTANRLEY BARCELOS PINTO,27/04 ANÁLIA-Estanrley Saida: 16:35,PROBS,27/04/2023,ANALIA,NaN,13:00,17:00,04:00,tarde,SEMANA - TARDE - C.CIRURGICO,semana
4,[27/04/2023 16:14:13] Fabiana Soares Gama: 27/...,27/04/2023,FABIANA SOARES GAMA,27/04 Villa Fabiana Gama saída as 16:00,PROBS,27/04/2023,VILLA,NaN,13:00,16:00,03:00,tarde,SEMANA - TARDE - C.CIRURGICO,semana


In [42]:
len(df)

31

# web scraping e limpeza

In [7]:
#########2
#capturando registros
unidades = ['ADMINISTRACAO','ALPHA','ANALIA','AVICCENA',
            'ITAIM','JABAQUARA','MATERNIDADE','MORUMBI',
            'OSASCO','VILLA','VNS','SANTA ISABEL']

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando resido do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²

    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2023" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    return tabela
    
tabela = funcao()
tabela

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data
0,1,ITAIM,SEMANA - MANHA - C.CIRURGICO,1,1,07:00,13:00,1,01/04/2023
1,1,ITAIM,SEMANA - TARDE - C.CIRURGICO,1,1,13:00,15:30,2,01/04/2023
2,2,ITAIM,SEMANA - 6HS,1,2,06:00,07:00,1,02/04/2023
3,2,ITAIM,SEMANA - MANHA - C.CIRURGICO,1,2,07:00,13:00,2,02/04/2023
4,3,JABAQUARA,SEMANA - MANHA - C.CIRURGICO,1,3,07:00,13:00,1,03/04/2023
5,3,JABAQUARA,SEMANA - TARDE - C.CIRURGICO,1,3,13:00,19:00,2,03/04/2023
6,3,JABAQUARA,SEMANA - NOTURNO - PLANTAO,1,3,19:00,07:00,3,03/04/2023
7,8,ITAIM,SEMANA - MANHA - C.CIRURGICO,2,1,07:00,13:00,1,08/04/2023
8,8,ITAIM,SEMANA - TARDE - C.CIRURGICO,2,1,13:00,19:00,2,08/04/2023
9,8,ITAIM,SEMANA - NOTURNO - SAIDA,2,1,19:00,22:00,3,08/04/2023


# alterando horario

In [44]:
len(df_backup)

31

In [45]:
count = 0

In [46]:
from selenium.webdriver.common.action_chains import ActionChains

action_chains = ActionChains(browser)

In [51]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print("nome:", i)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a/span[1]').click()
    #browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div/div[1]/div[3]/professionals/div/a/span[2]/b').click()
    sleep(2)
    
    browser.find_element("xpath", '/html/body/div[8]/div/input').send_keys(i)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li/div').click()

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))

    
    tabela = funcao()    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        
        dia_sel = tabela[(tabela['Unidade'] == df_nome['unidade'][t]) & 
                 (tabela['data'] == df_nome['data'][t]) & 
                 (tabela['Periodo'] == df_nome["setor"][t])]
        dia_sel.reset_index(drop=True, inplace=True)
        dia_sel
        
        if len(dia_sel) == 0:
            print(count, "-> PROB")
            df_backup['controle'][count] = "PROB"
        else:
            #abrindo/selecionando periodo conforme localização
            elemento = browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li')
            elemento.click()
            
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
            
            #Inicio do plantão
            browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df_nome['ENTRADA'][t])
            sleep(1)
            #print('hora de entrada:', df_nome['ENTRADA'][t], '\nhora de saida:', df_nome['SAIDA'][t])
            #Duração do plantão
            browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df_nome['DURAÇÃO'][t])
            #print('duração do plantão', df_nome['DURAÇÃO'][t])
            
            #salvar plantão
            sleep(1)
            browser.find_element("xpath", '/html/body/div[5]/div/div[1]/div[2]/button[5]').click()
            browser.find_element("xpath", '/html/body/div[5]/div/div[1]/div[2]/button[5]').click()
            browser.find_element("xpath", '/html/body/div[5]/div/div[1]/div[2]/button[5]').click()
            sleep(4)
            print(count, "-> OK")
            df_backup['controle'][count] = "OK"
        
        count += 1 
    print("\n")

nome: EMERSON DOMINGOS DA COSTA
28/04 Osasco Emerson saída 20:20
2 -> PROB


nome: ESTANRLEY BARCELOS PINTO
27/04 ANÁLIA-Estanrley Saida: 16:35
3 -> OK


nome: FABIANA SOARES GAMA
27/04 Villa Fabiana Gama saída as 16:00
4 -> OK


nome: FERNANDO RIBEIRO SA
28/04 VNS Ribeiro saída 15:15
5 -> OK


nome: JESSICA LIDIA DE SOUZA
27/04 Villa - Jessica entrada 11:00  (  À pedido da coordenação ) 
6 -> OK


nome: JOSE CARLOS TUNA SCARPA
27/04 ANALIA José Carlos Saída 17:25
7 -> OK


nome: JOSE MARCOS VIEIRA DA SILVA
27/04 - Itaim José Marcos saída 13:20
8 -> OK
28/04 - Itaim José Marcos saída 10:15
9 -> OK


nome: JULIANA AKEMI FUNABASHI
27/04 ANÁLIA - Akemi saída 16:50
10 -> OK


nome: JULIANO MOISES TOBAL
27/04 Osasco - Juliano Tobal saída 17:45
11 -> OK


nome: KAREN MANOELA ROSA
27/04 Santa Isabel K Rosa saída 17:10
12 -> OK


nome: KAREN YUMI MIYASHIRO
27/04 VILLA - Yumi saída 17:20h
13 -> PROB


nome: LUCCAS VARELA GODOY
27/4 Osasco Luccas saída 16:10
14 -> OK


nome: LUCIANO ANGELO RICHE

In [28]:
count += 1
df_backup['controle'][count] = "OK"

In [26]:
browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li').click()

In [85]:
df_backup['nome'][df_backup['nome'] == 'CAIO CESAR LOBO SAMPAIO'] = 'CAIO TULIO DUARTE'

In [19]:
dia_sel['Semana'][0]

4

In [33]:
df_backup['controle'][count] = "OK"
count += 1 

In [39]:
df_backup['controle'].value_counts()

OK       137
PROB      38
PROBS      1
Name: controle, dtype: int64

In [89]:
df_backup['controle'][df_backup['controle'] == 'PROB'] = 'PROBS'

In [ ]:
df_backup.iloc[count:count+3]

In [ ]:
df_backup

In [ ]:
acount

In [ ]:
/html/body/div[8]/div

# Salvando dataframe

In [52]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))
df_corrigidos.reset_index(drop=True, inplace=True)

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

probs ok: 25
probs: 6


In [53]:
df_corrigidos.to_excel("df_gabs2.xlsx", index=False)
df_prob.to_excel("df_giov2.xlsx", index=False)

# Fim Periodo 1

In [42]:
browser.quit()

In [54]:
print("Registros dos dias:", sorted(list(df_prob["dia"].unique())))
print("quantidade de registros:", len(df_prob), "=",
      str(round((len(df_prob)/len(df_backup))*100, 2)) + "% de todos os registros feitos")
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

Registros dos dias: ['27/04/2023', '28/04/2023']
quantidade de registros: 6 = 19.35% de todos os registros feitos
############################## 

ALEXANDRE BERNARDINO ==========
27/04 - Aviccena - Alexandre - entrada 05:30


EMERSON DOMINGOS DA COSTA ==========
28/04 Osasco Emerson saída 20:20


KAREN YUMI MIYASHIRO ==========
27/04 VILLA - Yumi saída 17:20h


LUCIANO ANGELO RICHETTI ==========
27/04 Osasco Luciano Saída  15:10


RONALD FIGUEIREDO GOMES ==========
28/04 JABAQUARA- Ronald saida 15:50


SHEILA ITIROCO MONTE ==========
27/04 SANTA Sheila saída 14:40




In [ ]:
#CAPTURANDO NOME DE TODOS PROFISSIONAIS ========================================================
browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a').click() #barra de nome

html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML') #
soup_nome = BeautifulSoup(html, 'html.parser')
#soup_nome

NOMES = []
for i in soup_nome.find_all(class_ ="select2-results-dept-0 select2-result select2-result-selectable"):
    #print(i.get_text())
    NOMES.append(i.get_text())
#del(NOME[0])

NOMES

# detecção problemas de inserção de registros

In [ ]:
dias_fds = ['07','08','14','15','21','22']

for semana in tabela['Semana'].unique(): 
    tabela_semana = tabela[tabela['Semana'] == semana] # _isolar semana_
    #print('SEMANA', semana, '='*50)

    for dia in tabela_semana['Dia'].unique(): #verifique cada dia
        #print('DIA', dia, '='*40)
        tab_dia = tabela_semana[tabela_semana['Dia'] == str(dia)] #tabela dia XX # _isolar dia_
        tab_dia.reset_index(drop=True, inplace=True) #resetando index

        #ENCONTRANDO REGISTRO UNICOS, QUE NÃO SEJAM FINAL DE SEMANA COM PERIODOS MAIOR QUE 6H
        if len(tab_dia) == 1: #se contem apenas um registro
            if dia in dias_fds and int(tab_dia['Saida'][0][:2]) - int(tab_dia['Entrada'][0][:2]) <= 6:
                continue #pule 
            elif (int(tab_dia['Dia'][0]) <= 25 and semana == 1) or (int(tab_dia['Dia'][0]) == 1 and semana == 5):
                continue #pule 
            else: #se não for final de semana
                print(dia, '- valor unico:', '[', 
                      tab_dia['Entrada'][0][:2],'...',tab_dia['Saida'][0][:2],
                      '-',tab_dia['Unidade'][0],'-',tab_dia['Periodo'][0],']') #indique
                #print(int(tab_dia['Saida'][0][:2]), int(tab_dia['Entrada'][0][:2]))

        #ENCONTRANDO DIAS QUE NÃO BATEM OS HORARIOS DE ENTRADA E SAIDA DOS TURNOS
        for t in range(len(tab_dia)): #para cada registro da tabela
            if t != 0: #em excessão do primeiro
                if tab_dia['Entrada'][t] != tab_dia['Saida'][t-1]: 
                    #verifique se a hora do inicio do plantão é igual a fim do plantão anterior
                    print(dia, '- erro entre os horarios:', 
                          '[', tab_dia['Entrada'][t], '...',tab_dia['Saida'][t-1],']') #se for, indique                        

        #ENCONTRANDO DIAS COM MAIS DE UMA UNIDADE REGISTRADA
        unidades_presentes = tab_dia['Unidade'].unique()
        if len(unidades_presentes) != 1:
            print(dia, '- unidade diferentes:', unidades_presentes)

        #ENCONTRANDO FINAIS DE SEMANA REGISTRADO COMO SEMANA
        if dia in dias_fds:
            for registro in range(len(tab_dia)):
                if 'FDS' not in tab_dia['Periodo'][registro]:
                    if (int(tab_dia['Dia'][registro]) == 31 and semana == 1) or (int(tab_dia['Dia'][registro]) <= 3 and semana == 5):
                        continue
                    else:
                        print(tab_dia['Dia'][registro], "- fim de semana errado:",
                              '[', tab_dia['Periodo'][registro], ']')

        if dia not in dias_fds:
            for registro in range(len(tab_dia)):
                if 'SEMANA' not in tab_dia['Periodo'][registro]:
                    if (int(tab_dia['Dia'][registro]) == 31 and semana == 1) or (int(tab_dia['Dia'][registro]) <= 3 and semana == 5):
                        continue
                    else:
                        print(tab_dia['Dia'][registro], '- dia da semana errado:',
                              '[', tab_dia['Periodo'][registro], ']')

        for j in range(len(tab_dia)):
            if tab_dia['Periodo'][j] == 'FDS - PLANTAO':
                if int(tab_dia['Entrada'][j][0:2]) < 18:
                    print(tab_dia['Dia'][j], '- [FDS - PLANTÃO] registrado a tarde -'
                          '[', tab_dia['Entrada'][j], '...',tab_dia['Saida'][j],']')
            if tab_dia['Periodo'][j] == 'FDS - SAIDA':
                if int(tab_dia['Entrada'][j][0:2]) > 18:
                    print(tab_dia['Dia'][j], '- [FDS - SAIDA] registrado a noite -'
                          '[', tab_dia['Entrada'][j], '...',tab_dia['Saida'][j],']')


In [ ]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print("nome:", i)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a/span[1]').click()
    sleep(2)
    browser.find_element("xpath", '/html/body/div[7]/div/input').send_keys(i)
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li/div').click()

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
    
    tabela = funcao()
    tabela
    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(count)
        print(df_nome['mensagem'][t])
        
        dia_sel = tabela[(tabela['Unidade'] == df_nome['unidade'][t]) & 
                 (tabela['data'] == df_nome['data'][t]) & 
                 (tabela['Periodo'] == df_nome["setor"][t])]
        dia_sel.reset_index(drop=True, inplace=True)
        dia_sel
        
        if len(dia_sel) == 0:
            print("PROB")
            df_backup['controle'][count] = "PROB"
        else:
            print("OK")
            df_backup['controle'][count] = "OK"
        count += 1 